<h1>Extras - Basic Low Pass Filtering in Python</h1>

Some extra material for the exercise in Lab 5 of CM2015 Project Carrier Course, Part 1 at KTH Royal Institute of Technology.

This Jupyter notebook uses Python, Numpy, Matplotlib, and sometimes SciPy. To use these packages we need to import them first.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal as sg

In [ ]:
# Here we borrow a signal from LPFilter.ipynb

signal = [1.070, 1.070, 1.073, 1.077, 1.077, 1.076, 1.076, 1.076, 1.076, 1.076, 1.076, 1.076, 1.076, 1.076, 1.078,
          1.082, 1.085, 1.087, 1.089, 1.091, 1.095, 1.099, 1.103, 1.107, 1.109, 1.109, 1.111, 1.118, 1.133, 1.162,
          1.215, 1.296, 1.409, 1.547, 1.709, 1.882, 2.056, 2.220, 2.367, 2.478, 2.564, 2.644, 2.713, 2.768, 2.812,
          2.847, 2.873, 2.893, 2.908, 2.918, 2.925, 2.928, 2.923, 2.912, 2.903, 2.893, 2.881, 2.863, 2.841, 2.813,
          2.782, 2.747, 2.706, 2.660, 2.613, 2.562, 2.503, 2.435, 2.358, 2.275, 2.188, 2.101, 2.016, 1.935, 1.861,
          1.798, 1.743, 1.701, 1.669, 1.638, 1.610, 1.585, 1.561, 1.538, 1.515, 1.494, 1.473, 1.453, 1.431, 1.413,
          1.396, 1.377, 1.359, 1.339, 1.320, 1.300, 1.282, 1.265, 1.249, 1.234, 1.219, 1.206, 1.196, 1.189, 1.178,
          1.167, 1.156, 1.147, 1.137, 1.129, 1.123, 1.117, 1.112, 1.108, 1.106, 1.106, 1.103, 1.100, 1.096, 1.091,
          1.087, 1.084, 1.082, 1.081, 1.081]

S = np.array(signal + signal + signal)
X = np.arange(len(S)) / 125.0

# Padding

When we apply a FIR filter to a signal that is not infinite in both directions, we will end up with the problem of not having all values.  The example below illustrates the problem. Try to explain to yourself what is happening in the beginning of the filtered signal. Why does it look off?

We call this a border effect. A way to deal with this is padding. Padding essentially means that we invent values for the missing signal parts, extending it for the sake of the filter.

In [ ]:
def lfilter_padding(b, a, x, pad_width=0, mode='constant', **kwargs):
    """This function applies a causual filter but with padding before."""
    if pad_width > 0:
        x = np.pad(x, pad_width, mode=mode, **kwargs)
    
    y = sg.lfilter(b, a, x)
    return y[pad_width:-pad_width] if pad_width > 0 else y

# A simple moving average filter with length M
M = 75
h = np.ones(M)/float(M)  # The filter coefficients

# Apply the filter to the signal, with different padding modes
y1 = lfilter_padding(h,1,S, pad_width=125, mode='constant', constant_values=0)
y2 = lfilter_padding(h,1,S, pad_width=125, mode='edge')
y3 = lfilter_padding(h,1,S, pad_width=125, mode='reflect')
y4 = lfilter_padding(h,1,S, pad_width=125, mode='wrap')


fig = plt.figure(figsize=(14,6))
plt.axes().set_aspect(0.25)
plt.plot(X,S,'-')
plt.plot(X,y1,'-', label='zero padding')
plt.plot(X,y2,'-', label='edge padding')
plt.plot(X,y3,'-', label='reflect padding')
plt.plot(X,y4,'-', label='wrap padding')
plt.grid()
plt.legend()
plt.show()

In [ ]:
# Let's look at the different padding modes:

S_padded1 = np.pad(S, 125, mode='constant', constant_values=0)
S_padded2 = np.pad(S, 125, mode='edge')
S_padded3 = np.pad(S, 125, mode='mean')
S_padded4 = np.pad(S, 125, mode='reflect')
S_padded5 = np.pad(S, 125, mode='wrap')

X_padded = np.arange(len(S_padded1)) / 125.0 - 1.0

fig = plt.figure(figsize=(14,6))
plt.axes().set_aspect(0.25)
plt.plot(X[:125], S[:125],'-', label='signal')
plt.plot(X_padded[:126], S_padded1[:126],'--', label='zero padding')
plt.plot(X_padded[:126], S_padded2[:126],'--', label='edge padding')
plt.plot(X_padded[:126], S_padded3[:126],'--', label='mean padding')
plt.plot(X_padded[:126], S_padded4[:126],'--', label='reflect padding')
plt.plot(X_padded[:126], S_padded5[:126],'--', label='wrap padding')
plt.grid()
plt.legend()
plt.show()

In this case, the *wrap padding* is probably the best padding. But that is due to the periodic signal that we have and that our array contains an exact multiple of exactly one period. If that is not the case, other paddings might be better.

# Why a filter with bell shaped coefficients is useful

Again, let's go back to our noise from LPFilter.ipynb. We add artificial noise at 15 Hz and 40 Hz. Let's plot it again.

Our task is now to try to eliminate the noise using filtering.

In [ ]:
# Just plot the signal again and with the noise added

f=40 #Hz
N = 0.2*np.cos(2*np.pi*f*X)
f=15
N += 0.1*np.cos(2*np.pi*f*X)

# Draw the signal with and without noise
fig = plt.figure(figsize=(14,6))
ax = plt.axes()
ax.set_aspect(0.25)
plt.plot(X, S+N, '-', label='with noise')
plt.plot(X, S, '-', label='no noise')
ax.legend(loc='upper right')
plt.grid()
plt.show()

The noise that we added have frequncies of 40 Hz and 15 Hz. If we know that we have noise of only a single frequency, we can design a moving average specifically for those frequencies. A moving average of size $M_1 = \frac{125}{40} \approx 3$ and $M_2 = \frac{125}{15} \approx 8$ should do a good job, since they average over (almost) exactly one full noise period. This is the same reason as finding the right $M$ for question B in LPFilter.ipynb.

We can apply one filter after the other using convolution.  However, the combination of FIR filters is also a FIR filter with:

$$h_{a+b} = (h_a * h_b)$$

since convolution is associative. I.e., $ (y * (h_a * h_b)) = ((y * h_a) * h_b)$. 

**Proof**: Ask ChatGPT: "Show that discrete convolution is associative".

In the example below, we create two FIR filters (both moving average, one for filtering out the 15 Hz noise and one for the 40 Hz noise) and apply them one after each other (y1). Then we combine the two FIR filters into one combined filter (using convolution) and apply the combined filter to the signal (y2). Both will yield the same output. And the noise is gone in both cases.

In [ ]:
# Moving average filters
ha = np.ones(3)/3
hb = np.ones(8)/8

# Combine the two filters using convolution
hab = np.convolve(ha, hb)

# Both these are identical
y1 = np.convolve(np.convolve(S+N, ha, mode='same'), hb, mode='same')
y2 = np.convolve(S+N, hab, mode='same')

fig = plt.figure(figsize=(14,6))
ax = plt.axes()
ax.set_aspect(0.25)
plt.plot(X,y1,'-', label='2 FIRs')
plt.plot(X,y2,'-', label='1 combined FIR')
ax.legend(loc='upper right')
plt.grid()
plt.show()

However, the combined filter will only work for noise with 15 Hz and/or 40 Hz, but not as well if we have noise between 15 Hz and 40 Hz. In the example below, we have noise with $f=20$ Hz.

In [ ]:
# Noise with f=20 Hz.
f=20
N2 = 0.2*np.cos(2*np.pi*f*X)

y = np.convolve(S+N2, hab, mode='same')

fig = plt.figure(figsize=(14,6))
ax = plt.axes()
ax.set_aspect(0.25)
plt.plot(X,y,'-', label='2 FIRs')
ax.legend(loc='upper right')
plt.grid()
plt.show()

We can make a Bode plot of the filters to understand why they remove noise with specific frequencies. In particular around the 15 and 40 Hz marks, but not at the 20 Hz mark.

In [ ]:
# Bode plot of the combined FIR filter

wa, yfa = sg.freqz(ha, fs=125)
wb, yfb = sg.freqz(hb, fs=125)
wab, yfab = sg.freqz(hab, fs=125)

fig = plt.figure(figsize=(14,6))
plt.plot(wa, 20 * np.log10(abs(yfa)+1e-10))
plt.plot(wa, 20 * np.log10(abs(yfb)+1e-10))
plt.plot(wab, 20 * np.log10(abs(yfab)+1e-10))
plt.plot([15,15],[-100,5], 'k--', label='15 Hz')
plt.plot([20,20],[-100,5], 'k--', label='15 Hz')
plt.plot([40,40],[-100,5], 'k--', label='15 Hz')
plt.ylim(-100,5)
plt.grid()
plt.show()

**Question 1**: Create a moving average FIR filter for noise at 20 Hz in the same way as above. Combine this filter with the other two FIR filters above (ha and hb) using convolution. Then plot the filtered signal again.

**Question 2**: Also plot the combined FIR filter (i.e., `plt.plot(h)`). Include both the figure and the code in your answer. Doesn't it look very similar to the sinc or hann functions? What happens if we combine additional FIR filters? Maybe for all frequencies between 15 and 40 Hz?

In [ ]:
# Moving average filters
h3 = np.ones(3)/3
h8 = np.ones(8)/8

# Combine them
h = np.convolve(h3, h8)

# TODO: combine more moving average filters
#h = np.convolve(h, np.ones(5)/5)
# ...

# Find the frequency response of the combined filter
w, yf = sg.freqz(h, fs=125)

fig1, (ax1,ax2) = plt.subplots(1,2,figsize=(18, 5))
ax1.plot(h, label='h')
ax1.set_title('Filter coefficients')
ax1.grid()
ax2.set_adjustable('datalim')
ax2.plot(w, 20 * np.log10(abs(yf)+1e-10))
ax2.set_title('Frequency response')
ax2.grid()
plt.show()

# Thanks for today!